In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import os

## Exercise 2, part 2

We build a CNN where both the input and the output are images (dogs vs cats) https://www.kaggle.com/c/dogs-vs-cats

Goal: The dataloader adds random noise to the image, and the network task is to remove that noise, recovering the original image.

In [ ]:
!wget https://www.dropbox.com/s/z90tvet2q97n350/cats.npy

In [ ]:
%load_ext autoreload
%autoreload 2

### Dataloader

You don't need to write it, just look at it. For every picture of a cat, it adds a random noise to it.

In [ ]:
from cats_dataloader import CatsWithNoiseDataset

In [ ]:
# I divide 80% of images to be our training set and 20% our validation set

train_ds = CatsWithNoiseDataset('cats.npy',0,800)
valid_ds = CatsWithNoiseDataset('cats.npy',800,1000)

In [ ]:
valid_ds

In [ ]:
len(train_ds)

In [ ]:
x, y = train_ds[100]
print(x.shape,y.shape)

fig,ax = plt.subplots(1,2,figsize=(6,3),dpi=150)

ax[0].imshow(y[0],cmap='gist_yarg',vmin=0,vmax=1)
ax[1].imshow(x[0],cmap='gist_yarg',vmin=0,vmax=1)

for i in range(2):
    ax[i].set_axis_off()

plt.show()

In [ ]:
# batch_size = 4, memory problem otherwise (but play around with it)

training_dataloader = DataLoader(train_ds,batch_size=4,shuffle=True)
valid_dataloader = DataLoader(valid_ds,batch_size=20)

In [ ]:
for x,y in training_dataloader:
    print(x.shape,y.shape)
    break

### The model

In [ ]:
# Conv2d(input channel, output channel, convolutional filter size)
# padding needs to be tuned in a way that output and input coincide

conv_layer = nn.Conv2d(1,50,3,padding = 1)

print(x.shape, conv_layer(x).shape)

In [ ]:
# BatchNorm2d It takes as input a batch with shape (N, input_size) 
# and normalizes each "column" in the input batch to have 
# mean 0 and variance 1.

We need to build a model that takes the images as input and outputs an image of the same size.

You can find an example of a model that works below (you can try to build your own). I repeated each block 5 times. Blue boxes correspond to the sequence.

* Conv2d
* BatchNorm2d
* ReLU

<div>
<img src="model_example.jpeg" width="600"/>
</div>

In this way the model will learn the negative value of the noise. Thefore the output will be the sum of the input and the result of the model (a residual block).

In [ ]:
from model_denoise import Net

In [ ]:
# Argument indicates number of central layer blocks (5)
# output size 25 central layers

net = Net()

print(net)

In [ ]:
# Make sure the output is same size of the input..!!!

x.shape, net(x).shape

## Training and validation

In [ ]:
# The error is MSE, mean square error. We are regressing the correct pixel.

loss_func = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4) 

In [ ]:
def compute_loss(dataloader,net):
    
    loss = 0
    
    if torch.cuda.is_available():
        net.cuda()
    net.eval()
    
    n_batches = 0
    with torch.no_grad():
        for x,y in dataloader:
            n_batches+=1
            
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            pred = net(x)
            
            loss+= loss_func(pred,y).item()
            
    loss = loss/n_batches      
    return loss

In [ ]:
# Before training
compute_loss(valid_dataloader,net)

In [ ]:
# It takes some time... in the mean time read about google colab, for next exercises we could start CUDA from there

if os.path.exist('trained_model.pt'):
    net.load_state_dict(torch.load('trained_model.pt',map_location='cpu'))
else:
    n_epochs = 100

    validation_loss_vs_epoch = []

    if torch.cuda.is_available():
        net.cuda()

    pbar = tqdm( range(n_epochs) )

    for epoch in pbar:

        if len(validation_loss_vs_epoch) > 1:
            print('epoch',epoch,' val loss:'+'{0:.5f}'.format(validation_loss_vs_epoch[-1]) )

        net.train() # put the net into "training mode"
        for x,y in training_dataloader:
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()

            optimizer.zero_grad()
            pred = net(x)
            loss = loss_func(pred,y)
            loss.backward()
            optimizer.step()

        net.eval() #put the net into evaluation mode

        valid_loss =  compute_loss(valid_dataloader,net)

        validation_loss_vs_epoch.append(valid_loss)

        if len(validation_loss_vs_epoch)==1 or validation_loss_vs_epoch[-2] > validation_loss_vs_epoch[-1]:
            torch.save(net.state_dict(), 'trained_model.pt')

In [ ]:
# After training
compute_loss(valid_dataloader,net)

In [ ]:
# Check the result with a random validation idx
x, y = valid_ds[103]

fig,ax = plt.subplots(1,3,figsize=(9,3),dpi=150)

ax[2].imshow(y[0],cmap='gist_yarg',vmin=0,vmax=1)
ax[0].imshow(x[0],cmap='gist_yarg',vmin=0,vmax=1)

net.eval()
net.cpu()
predicted = net( x.unsqueeze(1) )[0][0].data.numpy()

ax[1].imshow(predicted,cmap='gist_yarg',vmin=0,vmax=1)

ax[0].set_title('Input',fontsize=12)
ax[1].set_title('Network output',fontsize=12)
ax[2].set_title('Target',fontsize=12)
for i in range(3):
    ax[i].set_axis_off()

plt.show()